In [3]:
import numpy as np

height = [60, 62, 65,  68,  70,  72,  75,  77,  80,  82]
weight = [100, 110, 120, 130, 140, 150, 160, 170, 180, 190]

np.corrcoef(height, weight)[0][1]


0.9990716178370008

In [4]:
import pandas as pd

df = pd.read_csv('../../datasets/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df.corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000
